# Misspelling detection and correction

**NOTE**: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](../ch4/1.ch4-setting-up-the-retrotech-dataset.ipynb) notebook.

In [1]:
import pandas as pd
import nltk
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
signal_sample=pd.read_json("../data/temp/signal_sample.json")

### Step 1: Tokenize queries and count word frequencies. 
Check word frequency distribut quantiles. The quantile will help decide cut off point for potential misspells and corrections. 

In [3]:
stop_words = set(stopwords.words('english'))
word_list = defaultdict(int)

for query in signal_sample["query_s"]:
    tokenizer = RegexpTokenizer(r'\w+') 
    tokened = tokenizer.tokenize(query.lower())
    
    for token in tokened:
        if token not in stop_words and len(token) > 3 and not token.isdigit():  #drop stopwords and digit only tokens
            # and only consider token length > 3, since hard to judge whether a very short token is misspelled or not
            word_list[token] += 1

In [4]:
np.quantile(np.array(list(word_list.values())), [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])

array([1., 1., 1., 1., 1., 2., 3., 5., 9.])

### Step 2: compute metadata needed for word matching. 
consider word with low count as misspelling condidates, with high count as correctly spelled candidates. 

In [5]:
misspell_candidates = []
correction_candidates = []
misspell_counts = []
correction_counts = []
misspell_length = []
correction_length = []
misspell_initial = []
correction_initial = []

for k, v in word_list.items():
    if v == 1:
        misspell_candidates.append(k)
        misspell_counts.append(v)
        misspell_length.append(len(k))
        misspell_initial.append(k[0])
    if v >= 9:
        correction_candidates.append(k)
        correction_counts.append(v)
        correction_length.append(len(k))
        correction_initial.append(k[0])

In [6]:
misspell_candidates_df = pd.DataFrame({"misspell":misspell_candidates, "misspell_counts":misspell_counts, "misspell_length":misspell_length,"initial":misspell_initial})
correction_candidates_df = pd.DataFrame({"correction":correction_candidates, "correction_counts":correction_counts, "correction_length":correction_length,"initial":correction_initial})

### Step 3: Find potential matches 
based on edit distance and whether word initial is the same or not. 

In [7]:
def good_match(len1, len2, edit_dist): #allow longer words have more edit distance
    match = 0
    min_length = min(len1, len2)
    if min_length < 8:
        if edit_dist == 1: match = 1
    elif min_length < 11:
        if edit_dist <= 2: match = 1
    else:
        if edit_dist == 3: match = 1
    return match

In [8]:
matches_candidates = pd.merge(misspell_candidates_df, correction_candidates_df, on="initial")
#join missepll list with correction list based on whether they share the same initials to reduce matching time. 
matches_candidates["edit_dist"] = matches_candidates.apply(lambda row: nltk.edit_distance(row.misspell,row.correction), axis=1)
matches_candidates["good_match"] = matches_candidates.apply(lambda row: good_match(row.misspell_length, row.correction_length, row.edit_dist),axis=1)

In [9]:
matches = matches_candidates[matches_candidates["good_match"] == 1].drop(["initial","good_match"],axis=1)

### Step 4: rank potential matched corrections 
based on edit distance and correction word frequency. shorter edit distance and higher word count will be prefered. only the top one correction is selected for final matching. 

In [10]:
matches.sort_values(by=['misspell', 'edit_dist', 'correction_counts'], ascending=[True, True, False])
matches_final = matches.groupby('misspell').first().reset_index()

In [11]:
matches_final.head(10)

,misspell,misspell_counts,misspell_length,correction,correction_counts,correction_length,edit_dist
0,accesorios,1,10,accessories,16,11,2
1,accwsories,1,10,accessories,16,11,2
2,aceer,1,5,acer,40,4,1
3,ameria,1,6,america,23,7,1
4,antanna,1,7,antenna,27,7,1
5,appe,1,4,apple,93,5,1
6,aquois,1,6,aquos,10,5,1
7,archam,1,6,arkham,19,6,1
8,asis,1,4,asus,53,4,1
9,atenna,1,6,antenna,27,7,1
